In [1]:
import pandas as pd

# Text Normalization: Russian Language

In [2]:
# This challenge builds upon research out of Google: https://arxiv.org/pdf/1611.00068.pdf
# Essentially, text-to-speech systems must accuractely disambiguate text which can be spoken 
# different ways depending on context e.g. 12:00 am might be pronounced as "Twelve A M".
# Research suggests that traditional RNN approaches to this NLP problem are problematic.
# RNN embeddings convey semantic similarity through proximity in a vector space.
# Text Normalization fails to convey the correct meaning if one of few acceptable 
# interpretations is not used.

In [3]:
DATA_DIR = '../data/'
with open(DATA_DIR + 'ru_train.csv', 'r', encoding='UTF8') as infile:
    data = infile.readlines()

In [4]:
data = [x.strip() for x in data]

In [5]:
cols=data[0].replace('"', '').split(',')
data = data[1:]

In [6]:
sent_col = [x.split(',')[0] for x in data]
tok_col = [x.split(',')[1] for x in data]
data = [",".join(x.split(',')[2:]) for x in data]
data = [x.split('","') for x in data]
data = [[x.replace('"', '') for x in row] for row in data]

In [7]:
ru_df = pd.DataFrame(data, columns=cols[2:])

In [8]:
ru_df['sentence_id'] = sent_col
ru_df['token_id'] = tok_col

In [9]:
ru_df.head(10)

,class,before,after,sentence_id,token_id
0,PLAIN,По,По,0,0
1,PLAIN,состоянию,состоянию,0,1
2,PLAIN,на,на,0,2
3,DATE,1862 год,тысяча восемьсот шестьдесят второй год,0,3
4,PUNCT,.,.,0,4
5,PLAIN,Оснащались,Оснащались,1,0
6,PLAIN,латными,латными,1,1
7,PLAIN,рукавицами,рукавицами,1,2
8,PLAIN,и,и,1,3
9,PLAIN,сабатонами,сабатонами,1,4


In [10]:
print("Roughly {:.2f} percent of training tokens require normalization.".format(100 * ru_df.query("before!=after").shape[0] / len(ru_df)))

Roughly 12.51 percent of training tokens require normalization.


## Simple Baselines: Dictionary Based Methods

In [11]:
# Just as in the Instacart Market Basket Analysis, we turn to simple dictionary based
# methods to establish a simple baseline to compare cost/benefit of the additional
# overhead of maintaining an ML system

In [12]:
trans_df = ru_df[['before', 'after']].query("before!=after").groupby('before', as_index=False)['after'].agg(lambda x:x.value_counts().index[0])

In [13]:
# As in the Instacart challenge, we select the value most frequently associated with the key

In [14]:
trans_dict = dict(zip(trans_df.before, trans_df.after))

In [15]:
trans_dict

{'': 'n a',
 '#': 'решетка',
 '$': 'доллар',
 '$ 1,3 млн.': 'одну целую и три десятых миллиона долларов сэ ш а',
 '$ 1,5 млн.': 'одну целую и пять десятых миллиона долларов сэ ш а',
 '$ 1,7 млрд.': 'одной целой и семи десятых миллиарда долларов сэ ш а',
 '$ 10 млн': 'десять миллионов долларов сэ ш а',
 '$ 115 млн': 'ста пятнадцати миллионов долларов сэ ш а',
 '$ 15,2 млн.': 'пятнадцати целых и двух десятых миллиона долларов сэ ш а',
 '$ 16 млн.': 'шестнадцати миллионов долларов сэ ш а',
 '$ 2,1 млрд.': 'двух целых и одной десятой миллиарда долларов сэ ш а',
 '$ 20': 'двадцати долларов сэ ш а',
 '$ 21,6 млн.': 'двадцати одной целой и шести десятых миллиона долларов сэ ш а',
 '$ 220 млн.': 'двухсот двадцати миллионов долларов сэ ш а',
 '$ 23 млрд': 'двадцати трех миллиардов долларов сэ ш а',
 '$ 27 миллионов': 'двадцати семи миллионов долларов сэ ш а',
 '$ 271 млн.': 'двухсот семидесяти одного миллиона долларов сэ ш а',
 '$ 27млн.': 'двадцати семи миллионов долларов сэ ш а',
 '$ 3': 'тре

In [ ]:
sub = pd.read_csv(DATA_DIR + 'ru_test.csv')

In [ ]:
sub['after'] = sub.before.replace(trans_dict)

### Model Attributes
*  Ignores: local context, POS tags, n-grams, general rules
*  Requires large training set

### Further Directions
*  Foreign Words & Transliteration
*  Acronyms
*  Tree-based models using POS and local context